## **<p style="text-align: center;">GoProtect</p>**

Сервис  “Мой Чемпион” помогает спортивным школам фигурного катания, тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

**Цель:**

Создать модель, помогающую находить элементы, которые могут быть успешно исполнены спортсменом на соревновании. 
Сервис будет прогнозировать прогресс и возможное выполнение тех или иных элементов программы по истории предыдущих выступлений и выполнения элементов на соревнованиях.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
from collections import Counter

In [4]:
tournament_scores_df_final = pd.read_csv('data/tournament_scores_df_final.csv')

tournament_scores_df_final

,id,total_score_id,title,decrease,base_score,goe,avg_score,1Lo,2Lo,3Lo,...,CSp,SEQ,USpB,Sp,A,Lo,S,Lz,T,1W
0,1,1,2A,NaN,3.3,0.66,3.96,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,3F+3Lo,NaN,10.2,-0.11,10.09,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,1,3Lz,NaN,0.0,-1.30,5.19,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,CCoSp4,NaN,3.5,0.56,4.06,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,FCSp4,NaN,3.2,0.64,3.84,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172153,352008,463407,CCoSp3,NaN,3.0,0.60,3.60,0,0,0,...,0,0,0,0,0,0,0,0,0,0
172154,352009,463407,2Lz,NaN,2.1,0.00,2.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
172155,352010,463407,2Lo,NaN,1.7,0.23,1.93,0,1,0,...,0,0,0,0,0,0,0,0,0,0
172156,352011,463407,StSq2,NaN,2.6,0.52,3.12,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
units_df = pd.read_csv('data/units.csv')

units_df

,id,color,school_id
0,9474,green,244.0
1,733,green,203.0
2,734,green,235.0
3,735,green,168.0
4,736,green,168.0
...,...,...,...
4591,10122,green,198.0
4592,5108,green,62.0
4593,5109,green,27.0
4594,5110,green,62.0


In [6]:
units_df['id'].value_counts().sum()

4596

In [7]:
tournaments_df = pd.read_csv('data/tournaments.csv')

tournaments_df

,id,date_start,date_end,origin_id
0,1,2090-11-29,2090-12-01,2.0
1,2,2091-03-06,2091-03-10,1.0
2,3,2090-10-05,2090-10-08,2.0
3,4,2090-10-18,2090-10-21,2.0
4,5,2090-10-21,2090-10-24,2.0
...,...,...,...,...
137,7114,2092-04-15,2092-04-20,1.0
138,7116,2091-10-24,2091-10-27,0.0
139,7117,2092-04-29,2092-05-03,0.0
140,7115,2092-04-23,2092-04-26,2.0


In [8]:
total_scores_df_final = pd.read_csv('data/total_scores_df_final.csv')

total_scores_df_final

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,place,segment_name,...,no_spinning_top,wrong_edge_push,repeat_jump,no_libela,plus_one_bonus,plus_two_bonus,plus_three_bonus,no_standing_position,skill_symbol,time_violation
0,442027,304,4785,47.20,43.47,102.70,59.23,0.0,1,короткая,...,0,0,0,0,0,0,0,0,0,0
1,442028,604,4785,45.40,46.71,101.19,54.48,0.0,2,короткая,...,0,0,0,0,0,0,0,0,0,0
2,442029,409,4785,44.00,46.82,99.20,52.38,0.0,3,короткая,...,0,0,0,0,0,0,0,0,0,0
3,442030,524,4785,46.20,44.86,91.84,47.98,-1.0,4,короткая,...,0,0,0,0,0,0,0,0,0,0
4,442031,412,4785,44.20,42.59,89.73,48.14,-1.0,5,короткая,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21290,462405,34979,7116,21.67,26.02,46.62,21.10,-0.5,20,произвольная,...,0,0,0,0,0,0,0,0,0,0
21291,462406,1995,7116,23.20,25.50,46.22,21.22,-0.5,21,произвольная,...,0,1,0,0,0,0,0,0,0,0
21292,462407,34841,7116,21.83,25.00,45.46,20.96,-0.5,22,произвольная,...,0,0,0,0,0,0,0,0,0,0
21293,462408,34980,7116,21.17,26.00,45.24,20.74,-1.5,23,произвольная,...,0,0,0,0,0,0,0,0,0,0


Я планирую обучить мультилейбл классификацию (одна модель для всех элементов),которая будет предсказывать выполнение элемента в следующих соревнованиях,исходя из данных прошлых соревнований.Сейчас будем подготавливать датафрейм с нужными нам колонками,так как колонок много и не все понадобятся.

In [9]:
# создание датафрейма с нужными колонками
need_columns = ['id', 'unit_id', 'tournament_id', 'segment_name']
total_scores = total_scores_df_final[need_columns].copy()

In [10]:
total_scores

,id,unit_id,tournament_id,segment_name
0,442027,304,4785,короткая
1,442028,604,4785,короткая
2,442029,409,4785,короткая
3,442030,524,4785,короткая
4,442031,412,4785,короткая
...,...,...,...,...
21290,462405,34979,7116,произвольная
21291,462406,1995,7116,произвольная
21292,462407,34841,7116,произвольная
21293,462408,34980,7116,произвольная


In [11]:
# переименуем колонку 'id' в 'total_score_id' 
total_scores.rename(columns={'id': 'total_score_id'}, inplace=True)

In [12]:
# переименуем колонку 'id' в 'unit_id'
units_df.rename(columns={'id': 'unit_id'}, inplace=True)

elements_df = pd.merge(units_df, total_scores, on='unit_id', how='left')

In [13]:
# переименуем колонку 'id' в 'tournament_id' 
tournaments_df.rename(columns={'id': 'tournament_id'}, inplace=True)

elements_df = pd.merge(elements_df, tournaments_df, on='tournament_id', how='left')

In [14]:
elements_df

,unit_id,color,school_id,total_score_id,tournament_id,segment_name,date_start,date_end,origin_id
0,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,2.0
1,9474,green,244.0,442094.0,4785.0,произвольная,2091-12-20,2091-12-24,2.0
2,733,green,203.0,5246.0,17.0,короткая,2090-12-09,2090-12-12,2.0
3,733,green,203.0,5271.0,17.0,произвольная,2090-12-09,2090-12-12,2.0
4,734,green,235.0,442856.0,4793.0,короткая,2091-12-09,2091-12-11,2.0
...,...,...,...,...,...,...,...,...,...
22493,10122,green,198.0,461320.0,6885.0,произвольная,2092-04-06,2092-04-08,0.0
22494,5108,green,62.0,NaN,NaN,NaN,NaN,NaN,NaN
22495,5109,green,27.0,NaN,NaN,NaN,NaN,NaN,NaN
22496,5110,green,62.0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
tournament_scores_df_final

,id,total_score_id,title,decrease,base_score,goe,avg_score,1Lo,2Lo,3Lo,...,CSp,SEQ,USpB,Sp,A,Lo,S,Lz,T,1W
0,1,1,2A,NaN,3.3,0.66,3.96,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,3F+3Lo,NaN,10.2,-0.11,10.09,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,1,3Lz,NaN,0.0,-1.30,5.19,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,CCoSp4,NaN,3.5,0.56,4.06,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,FCSp4,NaN,3.2,0.64,3.84,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172153,352008,463407,CCoSp3,NaN,3.0,0.60,3.60,0,0,0,...,0,0,0,0,0,0,0,0,0,0
172154,352009,463407,2Lz,NaN,2.1,0.00,2.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
172155,352010,463407,2Lo,NaN,1.7,0.23,1.93,0,1,0,...,0,0,0,0,0,0,0,0,0,0
172156,352011,463407,StSq2,NaN,2.6,0.52,3.12,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
elements_df = pd.merge(elements_df, tournament_scores_df_final, on='total_score_id', how='left')

In [17]:
missing_color_row = elements_df[elements_df['color'].isnull()]
missing_color_row

,unit_id,color,school_id,total_score_id,tournament_id,segment_name,date_start,date_end,origin_id,id,...,CSp,SEQ,USpB,Sp,A,Lo,S,Lz,T,1W
107657,2658,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# удалим строку, где пропуск в столбце color,тк пропуски везде
elements_df = elements_df.dropna(subset=['color'])

In [19]:
pd.set_option('display.max_columns', None)
elements_df

,unit_id,color,school_id,total_score_id,tournament_id,segment_name,date_start,date_end,origin_id,id,title,decrease,base_score,goe,avg_score,1Lo,2Lo,3Lo,4Lo,1Lz,2Lz,3Lz,4Lz,1F,2F,3F,4F,1A,2A,3A,1T,2T,3T,4T,1S,2S,3S,4S,CCoSp1,CCoSp2,CCoSp3,CCoSp4,CCoSpB,CCoSp,CoSpB,CoSp1,CoSp2,CoSp,ChSq1,ChSq,StSqB,StSq1,StSq2,StSq3,StSq4,LSpB,LSp1,LSp2,LSp3,LSp4,FCSpB,FCSp1,FCSp2,FCSp3,FCSp4,FCSp,FSSpB,FSSp1,FSSp2,FSSp3,FSSp4,FSSp,ChSpl1,ChSpl,CSSpB,CSSp1,CSSp2,CSSp3,CSSp4,CSSp,F,1Eu,COMBO,REP,SSpB,SSp1,SSp2,SSp3,SSp4,SSp,CCSpB,CCSp1,CCSp2,CCSp3,CCSp4,CCSp,CSpB,CSp1,CSp2,CSp3,CSp4,CSp,SEQ,USpB,Sp,A,Lo,S,Lz,T,1W
0,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,2.0,156595.0,3Lz+3T,NaN,10.1,1.18,11.28,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,2.0,156596.0,2A,NaN,3.3,0.85,4.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,2.0,156597.0,FCSp4,NaN,3.2,0.87,4.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,2.0,156598.0,3F,NaN,0.0,-1.36,4.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,2.0,156599.0,SSp4,NaN,2.5,0.61,3.11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173350,10122,green,198.0,461320.0,6885.0,произвольная,2092-04-06,2092-04-08,0.0,336180.0,CSSp1,NaN,1.9,0.00,1.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173351,5108,green,62.0

In [20]:
elements_df['segment_name'].value_counts()

segment_name
произвольная     111890
короткая          55383
элементы           3861
прыжки              852
интерпретация       123
любители             43
Name: count, dtype: int64

In [21]:
# заполним пропуски в столбце 'segment_name' значением 'любители'
elements_df['segment_name'] = elements_df['segment_name'].fillna('любители')

# заполним пропуски в столбцах 'date_start' и 'date_end' значением pd.NaT
elements_df['date_start'] = elements_df['date_start'].fillna(pd.NaT)
elements_df['date_end'] = elements_df['date_end'].fillna(pd.NaT)

# удалим ненужные столбцы
columns_to_drop = ['origin_id', 'id', 'title', 'decrease', 'base_score', 'goe', 'avg_score']
elements_df = elements_df.drop(columns=columns_to_drop)

In [22]:
missing_total_score_id_row = elements_df[elements_df['total_score_id'].isnull()]
missing_total_score_id_row

,unit_id,color,school_id,total_score_id,tournament_id,segment_name,date_start,date_end,1Lo,2Lo,3Lo,4Lo,1Lz,2Lz,3Lz,4Lz,1F,2F,3F,4F,1A,2A,3A,1T,2T,3T,4T,1S,2S,3S,4S,CCoSp1,CCoSp2,CCoSp3,CCoSp4,CCoSpB,CCoSp,CoSpB,CoSp1,CoSp2,CoSp,ChSq1,ChSq,StSqB,StSq1,StSq2,StSq3,StSq4,LSpB,LSp1,LSp2,LSp3,LSp4,FCSpB,FCSp1,FCSp2,FCSp3,FCSp4,FCSp,FSSpB,FSSp1,FSSp2,FSSp3,FSSp4,FSSp,ChSpl1,ChSpl,CSSpB,CSSp1,CSSp2,CSSp3,CSSp4,CSSp,F,1Eu,COMBO,REP,SSpB,SSp1,SSp2,SSp3,SSp4,SSp,CCSpB,CCSp1,CCSp2,CCSp3,CCSp4,CCSp,CSpB,CSp1,CSp2,CSp3,CSp4,CSp,SEQ,USpB,Sp,A,Lo,S,Lz,T,1W
922,5104,lime,198.0,NaN,NaN,любители,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1151,2992,green,257.0,NaN,NaN,любители,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1411,3159,green,38.0,NaN,NaN,любители,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1412,10123,green,49.0,NaN,NaN,любители,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1420,10125,green,93.0,NaN,NaN,любители,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171297,5103,green,62.0,NaN,NaN,любители,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173351,5108,green,62.0,NaN,NaN,любители,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173352,5109,gree

In [23]:
elements_df = elements_df.dropna(subset=['total_score_id'])

Я нашла строки с пропусками в total_score_id. Проверила, стоят ли пропуски во всех колонках этих строк,после подтверждения данного факта,удалила строки с пропусками в total_score_id из датафрейма.

In [24]:
# заполним пропуски в столбце total_score_id значением 0
elements_df['school_id'] = elements_df['school_id'].fillna(0)

In [25]:
elements_df.isnull().sum().sum()

0

In [26]:
elements_df

,unit_id,color,school_id,total_score_id,tournament_id,segment_name,date_start,date_end,1Lo,2Lo,3Lo,4Lo,1Lz,2Lz,3Lz,4Lz,1F,2F,3F,4F,1A,2A,3A,1T,2T,3T,4T,1S,2S,3S,4S,CCoSp1,CCoSp2,CCoSp3,CCoSp4,CCoSpB,CCoSp,CoSpB,CoSp1,CoSp2,CoSp,ChSq1,ChSq,StSqB,StSq1,StSq2,StSq3,StSq4,LSpB,LSp1,LSp2,LSp3,LSp4,FCSpB,FCSp1,FCSp2,FCSp3,FCSp4,FCSp,FSSpB,FSSp1,FSSp2,FSSp3,FSSp4,FSSp,ChSpl1,ChSpl,CSSpB,CSSp1,CSSp2,CSSp3,CSSp4,CSSp,F,1Eu,COMBO,REP,SSpB,SSp1,SSp2,SSp3,SSp4,SSp,CCSpB,CCSp1,CCSp2,CCSp3,CCSp4,CCSp,CSpB,CSp1,CSp2,CSp3,CSp4,CSp,SEQ,USpB,Sp,A,Lo,S,Lz,T,1W
0,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9474,green,244.0,442068.0,4785.0,короткая,2091-12-20,2091-12-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173346,10122,green,198.0,461320.0,6885.0,произвольная,2092-04-06,2092-04-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
173347,10122,green,198.0,461320.0,6885.0,произвольная,2092-04-06,2092-04-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [27]:
elements_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 172152 entries, 0 to 173350
Columns: 104 entries, unit_id to 1W
dtypes: float64(99), int64(1), object(4)
memory usage: 137.9+ MB


In [28]:
# найдем все колонки с типом данных float64
float_columns = elements_df.select_dtypes(include=['float64']).columns
float_columns

Index(['school_id', 'total_score_id', 'tournament_id', '1Lo', '2Lo', '3Lo',
       '4Lo', '1Lz', '2Lz', '3Lz', '4Lz', '1F', '2F', '3F', '4F', '1A', '2A',
       '3A', '1T', '2T', '3T', '4T', '1S', '2S', '3S', '4S', 'CCoSp1',
       'CCoSp2', 'CCoSp3', 'CCoSp4', 'CCoSpB', 'CCoSp', 'CoSpB', 'CoSp1',
       'CoSp2', 'CoSp', 'ChSq1', 'ChSq', 'StSqB', 'StSq1', 'StSq2', 'StSq3',
       'StSq4', 'LSpB', 'LSp1', 'LSp2', 'LSp3', 'LSp4', 'FCSpB', 'FCSp1',
       'FCSp2', 'FCSp3', 'FCSp4', 'FCSp', 'FSSpB', 'FSSp1', 'FSSp2', 'FSSp3',
       'FSSp4', 'FSSp', 'ChSpl1', 'ChSpl', 'CSSpB', 'CSSp1', 'CSSp2', 'CSSp3',
       'CSSp4', 'CSSp', 'F', '1Eu', 'COMBO', 'REP', 'SSpB', 'SSp1', 'SSp2',
       'SSp3', 'SSp4', 'SSp', 'CCSpB', 'CCSp1', 'CCSp2', 'CCSp3', 'CCSp4',
       'CCSp', 'CSpB', 'CSp1', 'CSp2', 'CSp3', 'CSp4', 'CSp', 'SEQ', 'USpB',
       'Sp', 'A', 'Lo', 'S', 'Lz', 'T', '1W'],
      dtype='object')

In [29]:
elements_df[float_columns] = elements_df[float_columns].astype('int64')

In [30]:
# преобразуем столбцы в формат datetime
elements_df['date_start'] = pd.to_datetime(elements_df['date_start'], errors='coerce')
elements_df['date_end'] = pd.to_datetime(elements_df['date_end'], errors='coerce')

In [31]:
# total_score_id является идентификатором выступления и не несет полезной информации для обучения модели.,использовался как ключ для джойна таблиц
elements_df = elements_df.drop(columns=['total_score_id'])

In [32]:
# удалим date_end (дата завершения),тк не добавляет полезной информации для модели, поскольку date_start (дата начала) уже содержит временную информацию о событии
elements_df = elements_df.drop(columns=['date_end'])

In [33]:
# сортируем данные по спортсменам и дате для обеспечения временного порядка для каждого спортсмена
elements_df = elements_df.sort_values(by=['unit_id', 'date_start'])

In [34]:
# считаем количество уникальных турниров для каждого спортсмена
tournament_counts = elements_df.groupby('unit_id')['tournament_id'].nunique()
tournament_counts.value_counts()

tournament_id
1     1103
2      595
3      431
4      323
5      228
6      176
7      118
8       92
9       83
10      55
11      48
12      30
14      25
13      25
15      17
17      10
16       9
20       8
18       8
19       6
23       2
21       1
Name: count, dtype: int64

У спортсменов с 1 соревнованием нет исторических данных для анализа. У спортсменов с 2 соревнованиями можно использовать одно соревнование как тренировочные данные и одно соревнование как тестовые данные,что тоже не подходит. Спортсмены с 3 и более соревнованиями можно использовать несколько предыдущих соревнований для создания признаков (лагов) и прогнозирования результатов. 

Данные разделим так: последнее соревнование для теста, предпоследнее для валидации, остальные для обучения.

In [35]:
# фильтруем спортсменов с менее чем четырьмя уникальными соревнованиями
filtered_units = tournament_counts[tournament_counts >= 4].index
elements_df = elements_df[elements_df['unit_id'].isin(filtered_units)]

In [36]:
exclude_columns = ['unit_id', 'color', 'school_id', 'tournament_id', 'segment_name', 'date_start']
element_columns = [col for col in elements_df.columns if col not in exclude_columns]

In [37]:
# создаем лаги для каждого элемента
lags = 3
lag_dfs = []

for lag in range(1, lags + 1):
    lagged_df = elements_df.groupby('unit_id')[element_columns].shift(lag)
    lagged_df.columns = [f'{col}_lag{lag}' for col in element_columns]
    lag_dfs.append(lagged_df)

elements_df = pd.concat([elements_df] + lag_dfs, axis=1)

In [38]:
# удаляем строки с пропусками после создания лагов
elements_df = elements_df.dropna()

In [39]:
# разделение данных 
feature_columns = [col for col in elements_df.columns if col not in element_columns + ['unit_id', 'date_start', 'tournament_id']]

X = elements_df[feature_columns]
y = elements_df[element_columns]

categorical_features = ['color', 'segment_name']

In [40]:
unique_value_columns = [col for col in y.columns if y[col].nunique() == 1]
print(f"Столбцы с одним уникальным значением: {unique_value_columns}")

Столбцы с одним уникальным значением: ['1W']


In [41]:
y = y.drop(columns=unique_value_columns)
element_columns = [col for col in element_columns if col not in unique_value_columns]

In [42]:
# выделим последнее соревнование для каждого спортсмена в тестовую выборку
test_idx = elements_df.groupby('unit_id')['date_start'].idxmax()
test_df = elements_df.loc[test_idx]

# исключим строки, которые попали в тестовую выборку
remaining_df = elements_df.drop(test_idx)

# выделим предпоследнее соревнование для каждого спортсмена в валидационную выборку
val_idx = remaining_df.groupby('unit_id')['date_start'].idxmax()
val_df = remaining_df.loc[val_idx]

# исключим строки, которые попали в валидационную выборку
train_df = remaining_df.drop(val_idx)

In [43]:
# разделим данные на признаки и целевые переменные
X_train = train_df[feature_columns]
X_val = val_df[feature_columns]
X_test = test_df[feature_columns]

y_train = train_df[element_columns]
y_val = val_df[element_columns]
y_test = test_df[element_columns]

In [44]:
# вычислим веса классов
def compute_class_weights(y):
    class_weights = {}
    for col in y.columns:
        counter = Counter(y[col])
        total = sum(counter.values())
        weights = {cls: total / count for cls, count in counter.items()}
        class_weights[col] = weights
    return class_weights

class_weights = compute_class_weights(y_train)

In [45]:
models = {}
for col in element_columns:
    weights = class_weights[col]
    model = CatBoostClassifier(
        iterations=100,
        depth=5,
        learning_rate=0.1,
        loss_function='MultiClass',
        class_weights=weights,
        random_seed=42
    )
    model.fit(
        X_train,
        y_train[col],
        cat_features=categorical_features,
        eval_set=(X_val, y_val[col]),
        verbose=False
    )
    models[col] = model

# оценка моделей на тестовой выборке 
f1_scores = {}
for col in element_columns:
    preds = models[col].predict(X_test)
    f1 = f1_score(y_test[col], preds, average='weighted')
    f1_scores[col] = f1
    print(f"F1-Score для модели {col} на тестовой выборке: {f1:.2f}")

F1-Score для модели 1Lo на тестовой выборке: 0.70
F1-Score для модели 2Lo на тестовой выборке: 0.68
F1-Score для модели 3Lo на тестовой выборке: 0.87
F1-Score для модели 4Lo на тестовой выборке: 0.91
F1-Score для модели 1Lz на тестовой выборке: 0.73
F1-Score для модели 2Lz на тестовой выборке: 0.69
F1-Score для модели 3Lz на тестовой выборке: 0.71
F1-Score для модели 4Lz на тестовой выборке: 0.91
F1-Score для модели 1F на тестовой выборке: 0.36
F1-Score для модели 2F на тестовой выборке: 0.67
F1-Score для модели 3F на тестовой выборке: 0.81
F1-Score для модели 4F на тестовой выборке: 0.93
F1-Score для модели 1A на тестовой выборке: 0.52
F1-Score для модели 2A на тестовой выборке: 0.47
F1-Score для модели 3A на тестовой выборке: 0.89
F1-Score для модели 1T на тестовой выборке: 0.66
F1-Score для модели 2T на тестовой выборке: 0.68
F1-Score для модели 3T на тестовой выборке: 0.72
F1-Score для модели 4T на тестовой выборке: 0.90
F1-Score для модели 1S на тестовой выборке: 0.91
F1-Score для

In [46]:
print(f"Размер тренировочного набора: {X_train.shape[0]}")
print(f"Размер валидационного набора: {X_val.shape[0]}")
print(f"Размер тестового набора: {X_test.shape[0]}")

Размер тренировочного набора: 119914
Размер валидационного набора: 1264
Размер тестового набора: 1264


In [47]:
# проверка уникальности данных в наборах
assert len(set(X_train.index) & set(X_val.index)) == 0, "Есть пересечение между тренировочным и валидационным набором!"
assert len(set(X_train.index) & set(X_test.index)) == 0, "Есть пересечение между тренировочным и тестовым набором!"
assert len(set(X_val.index) & set(X_test.index)) == 0, "Есть пересечение между валидационным и тестовым набором!"

In [54]:
X_train['T_lag1'].value_counts()

T_lag1
0.0    119908
1.0         3
2.0         3
Name: count, dtype: int64

In [55]:
X_train['T_lag1'].value_counts()

T_lag1
0.0    119908
1.0         3
2.0         3
Name: count, dtype: int64

In [56]:
X_val['T_lag1'].value_counts()

T_lag1
0.0    1264
Name: count, dtype: int64

**<div style="text-align:center;">Итоговый вывод</div>**

В ходе реализации проекта была создана модель, помогающая находить элементы, которые могут быть успешно исполнены спортсменом на соревновании. Модель прогнозирует возможное выполнение тех или иных элементов программы на основе истории предыдущих выступлений и выполнения элементов на соревнованиях. Для реализации проекта была использована мультилейбл классификация с помощью CatBoostClassifier, одна модель для всех элементов. Модель показала хорошие результаты, использовалась метрика F1-Score.

Тем не менее, в процессе анализа выявились некоторые особенности: завышенная метрика для некоторых элементов,что связано с малым количеством данных для этих элементов и большим дисбалансом в данных. В результате при делении на тестовую выборку попадает малое количество разнообразных классов, что искажает метрику.

В датафрейме присутствуют спортсмены разного уровня. Одни участвуют в большом количестве соревнований и имеют достаточно данных для обучения модели, другие – новички, по данным которых модели сложно уловить закономерности.

Несмотря на это данные оказались очень интересными и разнообразными, что позволило выделить несколько перспективных направлений для дальнейшего развития проекта: разработать модель, которая предсказывает, какую ошибку чаще всего совершает спортсмен при выполнении каждого элемента. Создать модель для прогнозирования баллов, которые получит спортсмен за выполнение элементов. Анализировать динамику выступлений спортсмена и на основе этой информации рекомендовать, на какие элементы следует обратить внимание для улучшения результатов. 

Проект имеет большой потенциал для развития и может стать ценным инструментом для анализа и повышения спортивных результатов.